# 📓 **03 - RAG System & LLM Integration**

This notebook demonstrates the complete Retrieval-Augmented Generation (RAG) pipeline that combines semantic search with large language model capabilities. We'll show how to retrieve relevant products and generate intelligent recommendations using Mistral-7B.

### **Objectives of this notebook**

* **Implement intelligent retrieval** with domain detection and query parsing
* **Build a RAG pipeline** that combines FAISS vector search with LLM generation
* **Handle complex queries** with price filters, rating requirements, and multi-domain searches
* **Generate natural language recommendations** based on retrieved products

---

## 🔧 **System Architecture Overview**

The RAG system follows this workflow:

1. **Query Processing** → Parse natural language for domain hints, price filters, and rating requirements
2. **Semantic Retrieval** → Use FAISS to find relevant products based on embeddings
3. **Result Filtering** → Apply numeric constraints and domain-specific logic
4. **LLM Generation** → Use Mistral-7B to generate coherent product recommendations
5. **Response Formatting** → Present results in natural language with proper context

---

## 📚 **Import & Setup**

In [1]:
import os
import sys
import pandas as pd

# Add project root to Python path
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.insert(0, PROJECT_ROOT)

from src.retrieve import RAGRetriever
from src.generate_mistral import create_mistral_model, generate_answer

c:\Users\hasee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.4)
  from scipy.sparse import csr_matrix, issparse


---

## 🔍 **Intelligent Query Processing**

### **Domain Detection System**

The system automatically detects whether a query relates to Beauty, Electronics, or both domains using comprehensive keyword dictionaries:


In [2]:
# Example domain detection
rag = RAGRetriever()

test_queries = [
    "moisturizing cream for dry skin under 20 euros",
    "wireless headphones with noise cancellation",
    "gaming laptop and facial serum recommendations"
]

for query in test_queries:
    domain_hint = rag.get_domain_hint(query)
    print(f"Query: '{query}'")
    print(f"Domain Hint: {domain_hint}")
    print()

Query: 'moisturizing cream for dry skin under 20 euros'
Domain Hint: {'domain_hint': 'Beauty'}

Query: 'wireless headphones with noise cancellation'
Domain Hint: {'domain_hint': 'Electronics'}

Query: 'gaming laptop and facial serum recommendations'
Domain Hint: {'domain_hint': ['Beauty', 'Electronics']}




**How it works:**
- **Keyword Scoring**: Counts domain-specific terms in the query
- **Confidence Thresholding**: Requires minimum evidence before assigning domains
- **Multi-domain Support**: Handles queries spanning both Beauty and Electronics
- **Context Awareness**: Considers query length and clause structure

### **Natural Language Query Parsing**

The system extracts structured filters from natural language:


In [3]:
# Test query parsing
complex_query = "I want 3 affordable skincare products under 30 euros with at least 4-star rating"
parsed = rag.parse_query(complex_query)

print("🔍 Query Parsing Results:")
print(f"Original query: {complex_query}")
print(f"Price filter: {parsed['price_filter']}")
print(f"Rating filter: {parsed['rating_filter']}") 
print(f"Number of products: {parsed['num_products']}")

🔍 Query Parsing Results:
Original query: I want 3 affordable skincare products under 30 euros with at least 4-star rating
Price filter: ('<', 30.0)
Rating filter: ('>=', 4.0)
Number of products: 3



**Extracted filters include:**
- **Price ranges** ("under 30", "50-100", "over 200")
- **Rating requirements** ("above 4 stars", "highly rated")
- **Quantity specifications** ("show me 5 products", "recommend 3 items")

---

## 🎯 **Advanced Retrieval System**

### **Multi-Domain Query Decomposition**

For queries spanning multiple domains, the system intelligently splits them:

In [4]:
# Multi-domain query example
multi_domain_query = "gaming laptops and anti-aging serums under 100 euros"

print("Processing multi-domain query...")
results = rag.retrieve_with_decomposed_queries(
    query=multi_domain_query,
    domain_hint=["Electronics", "Beauty"]
)

print(f"Retrieved {len(results['dataframe'])} total products")
print("Domain distribution:")
print(results['dataframe']['domain'].value_counts())

Processing multi-domain query...
Retrieved 10 total products
Domain distribution:
domain
Electronics    5
Beauty         5
Name: count, dtype: int64



**Decomposition Process:**
1. **Clause Splitting**: Separates query into logical parts
2. **Domain Assignment**: Matches clauses to appropriate domains
3. **Confidence Scoring**: Determines how well each clause fits each domain
4. **Balanced Retrieval**: Ensures fair representation from all relevant domains

### **Intelligent Filtering & Ranking**

In [5]:

# Demonstrate filtering capabilities
filtered_query = "professional cameras over 500 euros with 4.5+ rating"
results = rag.retrieve(filtered_query, show_metadata=True)

print("📊 Filtered Results Analysis:")
print(f"Query: {filtered_query}")
print(f"Products found: {len(results['dataframe'])}")
print("\nTop results:")
display(results['dataframe'][['title', 'price', 'average_rating', 'domain']].head())

📊 Filtered Results Analysis:
Query: professional cameras over 500 euros with 4.5+ rating
Products found: 5

Top results:


,title,price,average_rating,domain
0,"Panasonic LUMIX G9 Mirrorless Camera, Micro Fo...",1097.99,4.7,Electronics
1,Panasonic LUMIX S5 Full Frame Mirrorless Camer...,1349.95,5.0,Electronics
2,Nikon D3500 DSLR Camera with 18-55mm and 70-30...,1174.00,4.5,Electronics
3,Nikon Z 7II Mirrorless Digital Camera Bundle w...,2603.99,4.9,Electronics
4,DJI RS 2 - 3-Axis Gimbal Stabilizer for DSLR a...,664.05,4.6,Electronics



**Filtering capabilities:**
- **Price constraints** with range support
- **Rating thresholds** with flexible operators
- **Domain-specific logic** (e.g., excluding camera accessories when searching for cameras)
- **Score boosting** based on title/category keyword matches


---

## 🤖 **LLM Integration & Response Generation**

### **Mistral-7B Model Setup**

In [6]:

# Load the language model
print("Loading Mistral-7B model...")
model, tokenizer = create_mistral_model()

if model and tokenizer:
    print("✅ Model loaded successfully!")
    print(f"Model device: {model.device}")
else:
    print("❌ Failed to load model")

Loading Mistral-7B model...
🔄 Loading model and tokenizer...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model and tokenizer loaded!
✅ Model loaded successfully!
Model device: cuda:0


### **Dynamic Prompt Engineering**

The system creates context-aware prompts that adapt to the number of retrieved products:
**Prompt Features:**
- **Dynamic Length Adjustment**: More products = shorter descriptions
- **Structured Product Format**: Consistent presentation of title, price, rating, description
- **Clear Instructions**: Explicit guidance on response format and requirements
- **Context Preservation**: Maintains query intent and user requirements

### **Response Generation & Quality Control**

In [7]:
# Test complete RAG pipeline
test_queries = [
    "best wireless earbuds under 50 euros",
    "moisturizers for sensitive skin with SPF",
    "gaming accessories and hair care products under 30 euros each"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    # Retrieve relevant products
    retrieval_results = rag.retrieve(query, show_metadata=False)
    products_df = retrieval_results['dataframe']
    
    print(f"📦 Retrieved {len(products_df)} products")
    
    # Generate LLM response
    if not products_df.empty:
        answer = generate_answer(
            query=query,
            retrieved_df=products_df,
            model=model,
            tokenizer=tokenizer,
            max_items=retrieval_results['requested_items']
        )
        print(f"🤖 GENERATED RESPONSE:\n{answer}")
    else:
        print("❌ No products found for this query")
    
    print(f"{'='*60}")


QUERY: best wireless earbuds under 50 euros


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📦 Retrieved 5 products
🤖 GENERATED RESPONSE:
Based on your request for the best wireless earbuds under 50 euros, I recommend the following products:

    1) Altec Lansing NanoPods - These truly wireless earbuds deliver powerful sound in a compact form. They offer clear highs and lows while blocking out background noise. Although they have a rating of only 3.5 stars, many users find their sound quality impressive for their price point.

    2) JLab Audio JBuds Air - With a price tag of around 39.99 EUR, these earbuds offer a comfortable fit, decent battery life, and good sound quality. Their rating is 4.1 stars, making them a popular choice among budget-conscious consumers.

    3) Anker Soundcore Life A2 NC - These earbuds offer active noise cancellation and a long battery life. While their design may not be as sleek as some higher-priced models, they produce clear and balanced sound. Their rating is 4.4 stars, which speaks to their popularity and reliability.

Although there are only 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📦 Retrieved 5 products
🤖 GENERATED RESPONSE:
Based on your request for moisturizers suitable for sensitive skin that also contain SPF, I would recommend the following two products:

    1. Product 4: Alitura Clay Mask - Organic Clay Mask for Men and Women
      This mask not only acts as a moisturizer but also contains SPF. Its organic formula ensures that it's gentle on sensitive skin. The addition of natural clay helps to draw out impurities while providing essential hydration. Its high rating of 4.4 indicates that many customers have had positive experiences using this product.

    2. Unfortunately, we do not have any other products on hand that meet your specifications, so the second recommendation is based on the assumption that you might consider a mask as an alternative to a traditional moisturizer. However, if you prefer a cream or lotion format, I strongly suggest revisiting this list at a later time when more suitable options become available.

QUERY: gaming accessories and 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📦 Retrieved 5 products
🤖 GENERATED RESPONSE:
Based on your request for gaming accessories and hair care products that cost less than 30 euros each, I recommend the following products:

Product 1: KISS Voguish Fantasy Ready-to-Wear Press-On/Glue-On Eyelashes
Price: 4.85 EUR
Rating: 3.7
This product is an excellent option for those who want to enhance their eyes without spending too much time or money at the salon. These press-on eyelashes can give you a glamorous look, and they come with various styles to suit different preferences. Although the rating is not very high, many users have reported positive experiences with this product.

Product 2: Laptop Phone TV Screen Keyboard Cleaner Brush Kit
Price: 11.99 EUR
Rating: 4.3
For those who spend long hours using electronics, keeping their screens clean is essential. This screen cleaning kit comes with various brushes designed to effectively remove dust, dirt, and grime from keyboards, laptops, phones, and TV screens. The set also includes 

---

## 📊 **System Performance & Quality Analysis**

### **Retrieval Quality Metrics**

In [11]:
import time
# Analyze retrieval performance
def evaluate_retrieval_quality(rag_system, test_cases):
    """Evaluate how well the system handles different query types"""
    
    results = []
    
    for query, expected_domains in test_cases:
        start_time = time.time()
        retrieval_result = rag_system.retrieve(query, show_metadata=True)
        end_time = time.time()
        
        df = retrieval_result['dataframe']
        actual_domains = df['domain'].unique().tolist() if not df.empty else []

        results.append({
            'query': query,
            'expected_domains': expected_domains,
            'actual_domains': actual_domains,
            'products_found': len(df),
            'response_time': end_time - start_time,
            'domain_match': set(expected_domains) == set(actual_domains)
        })
    
    return pd.DataFrame(results)

# Test cases
test_cases = [
    ("wireless bluetooth headphones", ["Electronics"]),
    ("face cream and serum", ["Beauty"]), 
    ("gaming laptop and skincare", ["Electronics", "Beauty"]),
    ("office laptop under 200 euros", ["Electronics"])
]

quality_df = evaluate_retrieval_quality(rag, test_cases)
print("Retrieval Quality Assessment:")
display(quality_df)

Retrieval Quality Assessment:


,query,expected_domains,actual_domains,products_found,response_time,domain_match
0,wireless bluetooth headphones,[Electronics],[Electronics],5,0.148818,True
1,face cream and serum,[Beauty],[Beauty],5,0.079211,True
2,gaming laptop and skincare,"[Electronics, Beauty]","[Beauty, Electronics]",10,0.233870,True
3,office laptop under 200 euros,[Electronics],[Electronics],5,0.153436,True


### **Response Quality Assessment**


In [ ]:
def assess_response_quality(query, generated_response, retrieved_products):
    """Basic assessment of response quality"""
    
    assessment = {
        'query': query,
        'response_length': len(generated_response),
        'products_mentioned': 0,
        'prices_included': False,
        'ratings_included': False,
        'reasoning_provided': False
    }
    
    # Check if products are mentioned
    for _, product in retrieved_products.iterrows():
        if product['text'] in generated_response:
            assessment['products_mentioned'] += 1
    
    # Check for price and rating mentions - FIXED VERSION
    assessment['prices_included'] = ('€' in generated_response) or ('eur' in generated_response.lower())
    assessment['ratings_included'] = any(str(rating) in generated_response for rating in retrieved_products['average_rating'].unique())
    assessment['reasoning_provided'] = any(word in generated_response.lower() for word in ['excellent option', 'because', 'since', 'due to', 'reason'])
    
    return assessment

# Example assessment
sample_query = "affordable skincare products"
sample_results = rag.retrieve(sample_query)['dataframe']
if not sample_results.empty and model and tokenizer:
    response = generate_answer(sample_query, sample_results, model, tokenizer, 3)
    quality = assess_response_quality(sample_query, response, sample_results)
    print("Response Quality Assessment:")
    for key, value in quality.items():
        print(f"  {key}: {value}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response Quality Assessment:
  query: affordable skincare products
  response_length: 1653
  products_mentioned: 1
  prices_included: True
  ratings_included: True
  reasoning_provided: True


---

## 💾 **System Outputs & Artifacts**

The RAG system produces:

1. **Structured Product Data**: Clean DataFrames with relevant products
2. **Natural Language Responses**: Coherent recommendations generated by Mistral-7B
3. **Query Analysis**: Parsed intent, domains, and filters
4. **Performance Metrics**: Retrieval times, result counts, quality assessments

---

## 🎯 **Real-World Applications**

This RAG system enables:

- **Intelligent E-commerce Search**: Natural language product discovery
- **Personalized Recommendations**: Context-aware suggestions based on user queries
- **Multi-domain Assistance**: Handling complex queries spanning categories
- **Filter-aware Retrieval**: Understanding price ranges and quality requirements
- **Conversational Commerce**: Natural dialogue about products and recommendations

---

## **Conclusion**

We've built a sophisticated RAG system that combines the precision of semantic search with the fluency of large language models. The system demonstrates:

✅ **Robust Query Understanding** with domain detection and filter extraction  
✅ **Efficient Semantic Retrieval** using FAISS vector similarity  
✅ **Intelligent Response Generation** with Mistral-7B  
✅ **Multi-domain Capabilities** handling complex, cross-category queries  
✅ **Production-ready Architecture** with proper error handling and performance monitoring  

This pipeline forms the foundation for intelligent product recommendation systems that can understand natural language, retrieve relevant options, and explain recommendations in a human-friendly manner.

**End of Notebook**